In [1]:
# 모듈 import
from mmcv import Config
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.datasets import build_dataloader, build_dataset   

In [2]:
classes = ("Backgroud", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
palette = [[0,0,0], [192,0,128], [0,128,192], [0,128,64], [128,0,0], [64,0,128],
           [64,0,192] ,[192,128,64], [192,192,128], [64,64,128], [128,0,192]]

# config file 들고오기
cfg_file = 'fcn_hr18_512x512_20k_voc12aug'
cfg = Config.fromfile(f'./configs/visual_trash_seg/merge/{cfg_file}.py')

# root = '../input/data/'

# dataset config 수정
# cfg.data.train.classes = classes
# cfg.data.train.data_root = root
# cfg.data.train.img_dir = 'train/img/'
# cfg.data.train.ann_dir = 'train/ann_rgb/'
# cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512)

# cfg.data.test.classes = classes
# cfg.data.test.data_root = root
# cfg.data.test.img_dir = 'test/img/'
# cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)

# cfg.data.samples_per_gpu = 4

# cfg.seed = 1995
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{cfg_file}'

# cfg.model.roi_head.bbox_head.num_classes = 10
# cfg.model.bbox_head.num_classes = 10

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.checkpoint_config = dict(max_keep_ckpts=10, interval=1)

In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

2021-10-21 08:55:15,513 - mmseg - INFO - Loaded 2617 images


In [4]:
# dataset 확인
datasets[0]

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_segmentor(cfg.model)
model.init_weights()

/opt/ml/segmentation/mmsegmentation/mmseg/models/backbones/hrnet.py:318: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2021-10-19 05:10:56,247 - mmseg - INFO - initialize HRNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://msra/hrnetv2_w48'}
2021-10-19 05:10:56,248 - mmcv - INFO - load model from: open-mmlab://msra/hrnetv2_w48
2021-10-19 05:10:56,251 - mmcv - INFO - Use load_from_openmmlab loader
2021-10-19 05:10:56,852 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: incre_modules.0.0.conv1.weight, incre_modules.0.0.bn1.weight, incre_modules.0.0.bn1.bias, incre_modules.0.0.bn1.running_mean, incre_modules.0.0.bn1.running_var, incre_modules.0.0.bn1.num_batches_tracked, incre_modules.0.0.conv2.weight, incre_modules.0.0.bn2.weight, incre_modules.0.0.bn2.bias, incre_modules.0.0.bn2.running_mean, incre_mod

In [6]:
# 모델 학습
# wandb.init(project='Object_Detection', entity='jujoo')
train_segmentor(model, datasets[0], cfg, distributed=False, validate=False)

2021-10-19 05:11:16,927 - mmseg - INFO - Start running, host: root@d40555fd3b37, work_dir: /opt/ml/segmentation/mmsegmentation/work_dirs/ocrnet_hr48_512x1024_160k_coco
2021-10-19 05:11:16,928 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                

In [ ]:
!python ./tools/analysis_tools/analyze_logs.py plot_curve ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json --keys loss --out losses.png

plot curve of ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json, metric is loss
save curve to: losses.png


In [8]:
!python ./tools/analysis_tools/analyze_logs.py plot_curve ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json --keys loss_cls loss_bbox --out losses.png

plot curve of ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json, metric is loss_cls
Traceback (most recent call last):
  File "./tools/analysis_tools/analyze_logs.py", line 180, in <module>
    main()
  File "./tools/analysis_tools/analyze_logs.py", line 176, in main
    eval(args.task)(log_dicts, args)
  File "./tools/analysis_tools/analyze_logs.py", line 55, in plot_curve
    f'{args.json_logs[i]} does not contain metric {metric}')
KeyError: './work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json does not contain metric loss_cls'
